# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-03-09 08:11:42] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=31869, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, tp_size=1, stream_interval=1, stream_output=False, random_seed=848279769, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0,

[2025-03-09 08:12:01 TP0] Init torch distributed begin.


[2025-03-09 08:12:01 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-09 08:12:01 TP0] Load weight begin. avail mem=78.30 GB
[2025-03-09 08:12:01 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-09 08:12:03 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-03-09 08:12:07 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-03-09 08:12:08] INFO:     Started server process [1359321]
[2025-03-09 08:12:08] INFO:     Waiting for application startup.
[2025-03-09 08:12:08] INFO:     Application startup complete.
[2025-03-09 08:12:08] INFO:     Uvicorn running on http://0.0.0.0:31869 (Press CTRL+C to quit)


[2025-03-09 08:12:08] INFO:     127.0.0.1:53162 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-09 08:12:09] INFO:     127.0.0.1:53164 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-09 08:12:09 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-09 08:12:11] INFO:     127.0.0.1:53166 - "POST /generate HTTP/1.1" 200 OK
[2025-03-09 08:12:11] The server is fired up and ready to roll!


Server started on http://localhost:31869


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-09 08:12:13 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-09 08:12:13 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 6.68, largest-len: 0, #queue-req: 0, 
[2025-03-09 08:12:14] INFO:     127.0.0.1:58528 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-03-09 08:12:14 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-09 08:12:14 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 56.34, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:15 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 59.15, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:15 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.01, gen throughput (token/s): 60.24, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:16] INFO:     127.0.0.1:58528 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-03-09 08:12:16] INFO:     127.0.0.1:58528 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-09 08:12:16 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 
It looks like we have a test. What can I help you with?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-03-09 08:12:16 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-09 08:12:16 TP0] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 66.06, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:17 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 93.48, largest-len: 0, #queue-req: 0, 
[2025-03-09 08:12:17] INFO:     127.0.0.1:58528 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-03-09 08:12:17 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-09 08:12:17 TP0] Decode batch. #running-req: 1, #token: 32, token usage: 0.00, gen throughput (token/s): 85.68, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:17 TP0] Decode batch. #running-req: 1, #token: 72, token usage: 0.00, gen throughput (token/s): 94.14, largest-len: 0, #queue-req: 0, 
[2025-03-09 08:12:17] INFO:     127.0.0.1:58528 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-03-09 08:12:18] INFO:     127.0.0.1:58538 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-09 08:12:18] INFO:     127.0.0.1:58538 - "POST /v1/batches HTTP/1.1" 200 OK
[2025-03-09 08:12:18 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-03-09 08:12:18 TP0] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 75.30, largest-len: 0, #queue-req: 0, 


Batch job status: validating...trying again in 3 seconds...
[2025-03-09 08:12:21] INFO:     127.0.0.1:58538 - "GET /v1/batches/batch_97d41f89-09b2-4b63-9947-b219ddd02b6d HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-03-09 08:12:21] INFO:     127.0.0.1:58538 - "GET /v1/files/backend_result_file-6ca58e78-6b1a-4b9b-b329-66bf19b9f01c/content HTTP/1.1" 200 OK


[2025-03-09 08:12:21] INFO:     127.0.0.1:58538 - "DELETE /v1/files/backend_result_file-6ca58e78-6b1a-4b9b-b329-66bf19b9f01c HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-03-09 08:12:21] INFO:     127.0.0.1:36950 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-09 08:12:21] INFO:     127.0.0.1:36950 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-09 08:12:21 TP0] Prefill batch. #new-seq: 7, #new-token: 210, #cached-token: 175, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-09 08:12:21 TP0] Prefill batch. #new-seq: 13, #new-token: 390, #cached-token: 325, token usage: 0.01, #running-req: 7, #queue-req: 0, 


[2025-03-09 08:12:21 TP0] Decode batch. #running-req: 20, #token: 1105, token usage: 0.05, gen throughput (token/s): 166.17, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:21 TP0] Decode batch. #running-req: 20, #token: 20, token usage: 0.00, gen throughput (token/s): 1726.84, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:31] INFO:     127.0.0.1:45140 - "GET /v1/batches/batch_e21c3055-c8c9-43fe-92fd-2f4961b4cbbd HTTP/1.1" 200 OK


[2025-03-09 08:12:34] INFO:     127.0.0.1:45140 - "GET /v1/batches/batch_e21c3055-c8c9-43fe-92fd-2f4961b4cbbd HTTP/1.1" 200 OK


[2025-03-09 08:12:37] INFO:     127.0.0.1:45140 - "GET /v1/batches/batch_e21c3055-c8c9-43fe-92fd-2f4961b4cbbd HTTP/1.1" 200 OK


[2025-03-09 08:12:40] INFO:     127.0.0.1:45140 - "GET /v1/batches/batch_e21c3055-c8c9-43fe-92fd-2f4961b4cbbd HTTP/1.1" 200 OK


[2025-03-09 08:12:43] INFO:     127.0.0.1:45140 - "GET /v1/batches/batch_e21c3055-c8c9-43fe-92fd-2f4961b4cbbd HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-03-09 08:12:46] INFO:     127.0.0.1:56352 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-09 08:12:46] INFO:     127.0.0.1:56352 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-09 08:12:47 TP0] Prefill batch. #new-seq: 22, #new-token: 80, #cached-token: 1130, token usage: 0.03, #running-req: 0, #queue-req: 0, 
[2025-03-09 08:12:47 TP0] Prefill batch. #new-seq: 112, #new-token: 3360, #cached-token: 2800, token usage: 0.03, #running-req: 22, #queue-req: 294, 


[2025-03-09 08:12:47 TP0] Decode batch. #running-req: 134, #token: 9405, token usage: 0.46, gen throughput (token/s): 201.63, largest-len: 0, #queue-req: 4866, 


[2025-03-09 08:12:48 TP0] Decode batch. #running-req: 134, #token: 14765, token usage: 0.72, gen throughput (token/s): 10899.51, largest-len: 0, #queue-req: 4866, 


[2025-03-09 08:12:48 TP0] Decode batch. #running-req: 134, #token: 20125, token usage: 0.98, gen throughput (token/s): 10802.44, largest-len: 0, #queue-req: 4866, 
[2025-03-09 08:12:48 TP0] Decode out of memory happened. #retracted_reqs: 16, #new_token_ratio: 0.5776 -> 1.0000
[2025-03-09 08:12:49 TP0] Prefill batch. #new-seq: 11, #new-token: 330, #cached-token: 275, token usage: 0.88, #running-req: 118, #queue-req: 4871, 
[2025-03-09 08:12:49 TP0] Prefill batch. #new-seq: 118, #new-token: 3540, #cached-token: 2950, token usage: 0.02, #running-req: 11, #queue-req: 4753, 


...

[2025-03-09 08:12:57 TP0] Decode batch. #running-req: 124, #token: 8085, token usage: 0.39, gen throughput (token/s): 6271.47, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:57 TP0] Decode batch. #running-req: 124, #token: 13045, token usage: 0.64, gen throughput (token/s): 9966.51, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:58 TP0] Decode batch. #running-req: 124, #token: 18005, token usage: 0.88, gen throughput (token/s): 9971.36, largest-len: 0, #queue-req: 0, 


[2025-03-09 08:12:59] INFO:     127.0.0.1:37558 - "GET /v1/batches/batch_75c6a408-c8ba-40a4-836f-ea2a5265caff HTTP/1.1" 200 OK


[2025-03-09 08:12:59] INFO:     127.0.0.1:37558 - "DELETE /v1/files/backend_input_file-372ad182-4917-4714-8ad3-c3d069edd18f HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)